In [ ]:
import random
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import kendalltau
from ast import literal_eval
import timeit

Questions, Dataset and motivation section:

Motivation:

League of legends is a multiplayer online battle arena video game developed and published by Riot Games which is one of the most popular game in the earth right now.The professional league is very exciting to watch that the 2018 League of Legends World Finals had nearly 100 million viewers. In the opposite, those presenters make poor predictions or ambiguous judgements on leads which gave views misleading on game real-time analysis all-time. Our goal is to dig deep in game real-time data, not only on the gold difference on two team but based on all other datas to predict the result of game, give viewers direct and precise feedback on real-time professional league game. This project could be further developed to website or tool to do function like real-time game analysis which all major analysis webside does not provide.

Questions:
Should team focus choose late game comp or early game comp on champion select in general based on winrate analysis?

If had a chance, should the team choose to destroy object, eliminate neutral monster or kill enemy players?

If we want to enjoy an entire series of game(best-of-five-sets format), how long does it take usual? We have to find the game length distributions.

DataSet:
Data set come from kaggle. Total 8 dataset and main dataset with 7620 rows and 58 columns. Contains all game in side status on each minutes from professional league game between 2015-2018 in the world. 

Input and Output:
Input is the game data after we transformate for model use, Output would be series of analysis graph and the blue team result(win or lose).

matchinfo = pd.read_csv('leagueoflegends/LeagueofLegends.csv')
gold = pd.read_csv('leagueoflegends/gold.csv')
kills = pd.read_csv('leagueoflegends/kills.csv')
info = pd.read_csv('leagueoflegends/matchinfo.csv')
structures = pd.read_csv('leagueoflegends/structures.csv')
monsters = pd.read_csv('leagueoflegends/monsters.csv')
columns = pd.read_csv('leagueoflegends/_columns.csv')

Detailed main data column element introduction:

columns

matchinfo.head(5)

matchinfo.to_pickle("leagueoflegends/match.pkl")

Save main data to pickle.

df = pd.read_pickle("leagueoflegends/match.pkl")

df.shape

df.head()

Main data datatypes:

df.dtypes

df.isnull().any()

Quality of cleaning:
1.Data Cleaning and type conversion activity:

We noticed that few columns are certain list attribute for each team(column 10-24 in our data frame). we should change the dtype of them. Also, because these attributes are list of minutes, it should has the length which is less than the value of game length or exactly the value of gamelength, we should check if that is the thing.


2.Missed value:
on column like Blue/Red Top,mid,Bottom, Support,Jungle, which is the role of the game and it contains player name and Blue/Red team tag. We would ignore it first because it is not the data we looking for this project. Detailed data cleaning activity were introduced on each section of data transformation below on each section.

3.Data summary statistics and interpretation.
on the end of data section.


df[['gamelength', 'bKills']].head(4)

len(df['golddiff'][0])

type(df['bKills'][0])

df['golddiff'] = df['golddiff'].apply(literal_eval)
df['goldblue'] = df['goldblue'].apply(literal_eval)
df['bKills'] = df['bKills'].apply(literal_eval)
df['bTowers'] = df['bTowers'].apply(literal_eval)
df['bInhibs'] = df['bInhibs'].apply(literal_eval)
df['bDragons'] = df['bDragons'].apply(literal_eval)
df['bBarons'] = df['bBarons'].apply(literal_eval)
df['bHeralds'] = df['bHeralds'].apply(literal_eval)
df['goldred'] = df['goldred'].apply(literal_eval)
df['rKills'] = df['rKills'].apply(literal_eval)
df['rTowers'] = df['rTowers'].apply(literal_eval)
df['rInhibs'] = df['rInhibs'].apply(literal_eval)
df['rDragons'] = df['rDragons'].apply(literal_eval)
df['rBarons'] = df['rBarons'].apply(literal_eval)
df['rHeralds'] = df['rHeralds'].apply(literal_eval)
df['goldblueTop'] = df['goldblueTop'].apply(literal_eval)
df['goldblueJungle'] = df['goldblueJungle'].apply(literal_eval)
df['goldblueMiddle'] = df['goldblueMiddle'].apply(literal_eval)
df['goldblueADC'] = df['goldblueADC'].apply(literal_eval)
df['goldblueSupport'] = df['goldblueSupport'].apply(literal_eval)
df['goldredTop'] = df['goldredTop'].apply(literal_eval)
df['goldredJungle'] = df['goldredJungle'].apply(literal_eval)
df['goldredMiddle'] = df['goldredMiddle'].apply(literal_eval)
df['goldredADC'] = df['goldredADC'].apply(literal_eval)
df['goldredSupport'] = df['goldredSupport'].apply(literal_eval)

Seems like gold diff is str. It should be the same situation other column data has, apply literal_eval function on those columns to fix it.

len(df['golddiff'][0])

len(df['bKills'][0])

type(df['bKills'][0])

df.dtypes

Game length distribution analysis below:

sns.set_style('whitegrid')
colors = sns.color_palette('hls', 8)

fig, ax = plt.subplots(1,2, figsize=(16,14))
fig.suptitle('Game Length Distributions', fontsize=24)
fig.subplots_adjust(top=0.9)
percentiles = np.array([25, 50, 75])
ptiles_gl = np.percentile(df['gamelength'], percentiles)
p1 = plt.subplot2grid((2,4), (0,0), colspan=1)
sns.boxplot(y=df['gamelength'], color=colors[5])
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.ylabel('Minutes', fontsize = 18, fontweight = 'bold')


p2 = plt.subplot2grid((2,4), (0,1), colspan=3)
x = np.sort(df['gamelength'])
y = np.arange(1, len(x) + 1) / len(x)
plt.plot(x,y, marker='.', linestyle='none', color=colors[5])
plt.plot(ptiles_gl, percentiles/100, marker='D', color='red', linestyle='none')
yvals = p2.get_yticks()
p2.set_yticklabels(['{:3.0f}%'.format(y*100) for y in yvals])
plt.yticks(fontsize=14)
plt.xticks(np.arange(0, 85, 5), fontsize=14)
plt.xlabel('Minutes', fontsize=18, fontweight = 'bold')
plt.ylabel('Percentage', fontsize=18, fontweight='bold')
plt.annotate('25% of games less than 32 minutes', xy=(32, .25), xytext=(37, .23), fontsize=18, 
             arrowprops=dict(facecolor='black'))
plt.annotate('50% of games less than 37 minutes', xy=(37, .5), xytext=(42, .48), 
             fontsize=18, arrowprops=dict(facecolor='black'))
plt.annotate('75% of games less than 42 minutes', xy=(42, .75), xytext=(47, .73), fontsize=18, 
             arrowprops=dict(facecolor='black'))


plt.tight_layout()
plt.show()

We find that about 50% game ends between 32 and 42 minuteswhich we has very less game ending early than 20 minutes or late than 60 minutes, we would use those facts for our future data analysis. It help us to figure out the data distribution too because all data were based on minutes ie. game time.

df['golddiff']

Gold difference vs winrate analysis:

we made a new data frame which stores only minute(real-time game length), Gold lead(Blue team) and the winrate based on the gold lead and the the game length on that time. Gold lead was calculated from the gold diff column , we made it from -10000 to 10000 with interval 500. Minute from 0 to 60 minutes with interval 5 to generate a heatmap on real-time gold difference and win-rate. Also three graph on early game(Minute <20),middle game(<20minute<40) and late game (40<minute<60)

dy = 500
dx = 5
df2= []
for index, row in df.iterrows():
    gdiff= [10000 if a > 10000 else -10000 if a < -10000 else int(int(a/dy)*dy) for a in row.golddiff]
    m= min(60+1,len(gdiff))        
    if row.bResult==1:
        for g in range(1, m): df2.append([int(int((g-1)/dx)*dx+dx), gdiff[g], 1])
        for g in range(1, m): df2.append([int(int((g-1)/dx)*dx+dx), -gdiff[g], 0])      
    elif row.rResult==1: 
        for g in range(1, m): df2.append([int(int((g-1)/dx)*dx+dx), -gdiff[g], 1])
        for g in range(1, m): df2.append([int(int((g-1)/dx)*dx+dx), gdiff[g], 0])
    else: print('Bad result'); continue  
df2= pd.DataFrame(df2); df2.columns= ['Time (min)', 'Gold Lead', 'Win Percent']

numgames= [int(x/10) for x in df2.groupby(['Time (min)']).size().reset_index(name='counts')['counts']]
df2= df2.groupby(['Time (min)', 'Gold Lead']).apply(lambda x: x['Win Percent'].sum()/len(x)).reset_index(name='Win Percent')
df2['Win Percent']= df2['Win Percent'].map(lambda x: int(x*100))
df3 = df2
df2= df2.pivot('Gold Lead', 'Time (min)', 'Win Percent') # GRAPH HEAT MAP
yticknames=['' if (k/500)%2==1 else str(int(k/1000.0))+'k' for k in range(-10000,10000+1,int(dy))]


df3

fig, ax = plt.subplots(2,2, figsize=(40,36))
fig.suptitle('Gold lead Winpercent Analysis', x=0.065, y=1.03, fontsize=24, fontweight='bold', 
             horizontalalignment='left')
fig.subplots_adjust(top=0.9)

percentiles = np.array([25, 50, 75])
ptiles_gl = np.percentile(df['gamelength'], percentiles)

plots = plt.subplot2grid((2,3), (0,0), colspan=1)
sns.heatmap(df2, vmin=0, vmax=100, yticklabels=yticknames, ax=plots, annot=True, cbar=False, fmt='g', cmap='viridis').invert_yaxis()
ax2= plots.twiny()
ax2.set_xlim(plots.get_xlim())
ax2.set_xticks([x+0.5 for x in range(int(dx/2),60,int(dx))]+[60])
ax2.set_xticklabels([int(x) for x in numgames])
ax2.set_xlabel(r'Number of Games')
ax2.grid(False)

plot2= plt.subplot2grid((2,3), (0,1), colspan=1)
plt.title('earlygame')
sns.barplot(x = df3['Gold Lead'][df3['Time (min)'].between(0, 32, inclusive=False)], y= df3['Win Percent'][df3['Time (min)'].between(0, 32, inclusive=False)])
plt.xticks(rotation='vertical',fontsize=10)
plot3= plt.subplot2grid((2,3), (1,0), colspan=1)
plt.title('middlegame')
sns.barplot(x = df3['Gold Lead'][df3['Time (min)'].between(32, 42, inclusive=False)], y= df3['Win Percent'][df3['Time (min)'].between(32, 42, inclusive=False)])
plt.xticks(rotation='vertical',fontsize=10)
plot4= plt.subplot2grid((2,3), (1,1), colspan=1)
plt.title('lategame')
sns.barplot(x = df3['Gold Lead'][df3['Time (min)'].between(42, 60, inclusive=False)], y= df3['Win Percent'][df3['Time (min)'].between(42, 60, inclusive=False)])
plt.xticks(rotation='vertical',fontsize=10)
plt.show()

We choose only data from 0 to 60 minutes because we find that very few games end after 60 minutes by game length analysis above.
Observation on gold diff on time vs winrate:
1.Gold is the direct attribute to reflect Leads
2.Importance of gold lead(gold diff) between two game become less as game length increase, but doesn’t fall off much.
3.Team with big gold lead in early period tends to win game straightly.

Conclusion: Early game > Late game

df.to_pickle("leagueoflegends/modified.pkl")

model = pd.read_pickle("leagueoflegends/modified.pkl")

matchinfo = pd.read_csv('leagueoflegends/LeagueofLegends.csv')
gold = pd.read_csv('leagueoflegends/gold.csv')
kills = pd.read_csv('leagueoflegends/kills.csv')
info = pd.read_csv('leagueoflegends/matchinfo.csv')
structures = pd.read_csv('leagueoflegends/structures.csv')
monsters = pd.read_csv('leagueoflegends/monsters.csv')
columns = pd.read_csv('leagueoflegends/_columns.csv')

We now using in-game data beside gold diffence and see how our model works on those data only. Instead of main dataset we would use more detailed data from other dataset and using address as our key for data reduction and transformation. We add a new column id which based on address column which is the website we could find the match data to match data on each game.

model['id'] = model['Address'].astype(str).str[-16:]
kills['id'] = kills['Address'].astype(str).str[-16:]
monsters['id'] = monsters['Address'].astype(str).str[-16:]
structures['id'] = structures['Address'].astype(str).str[-16:]
model.head()

monsters['Type'].unique()

Dragon was updated to Water ,Wind , Earth and Fire dragon after patch 6.9(2016 preseason), There is 5 column count for dragon now while one is expire. Have to clean those from our dataset.

old_dragon_id = monsters[ monsters['Type']=="DRAGON"]['id'].unique()
old_dragon_id

monsters = monsters[ ~monsters['id'].isin(old_dragon_id)]
monsters[monsters['Type']=="DRAGON"]

Same with kill.

kills = kills[ ~kills['id'].isin(old_dragon_id)]
kills = kills[ kills['Time']>0]

kills['Minute'] = kills['Time'].astype(int)

kills['Team'] = np.where( kills['Team']=="rKills","Red","Blue")
kills.head()

f = {'Time':['mean','count']}

killsGrouped = kills.groupby( ['id','Team','Minute'] ).agg(f).reset_index()
killsGrouped.columns = ['id','Team','Minute','Time Avg','Count']
killsGrouped = killsGrouped.sort_values(by=['id','Minute'])
killsGrouped.head(13)

choose only the data we need for future use.

Change datatype in structure and modify team column as the type we have in previous dataframe.

structures = structures[ ~structures['id'].isin(old_dragon_id)]
structures = structures[ structures['Time']>0]

structures['Minute'] = structures['Time'].astype(int)
structures['Team'] = np.where(structures['Team']=="bTowers","Blue",
                        np.where(structures['Team']=="binhibs","Blue","Red"))
structures2 = structures.sort_values(by=['id','Minute'])
structures2.head(13)

kills_structures = killsGrouped.merge(structures2[['id','Minute','Team','Time','Lane','Type']],
                                      on=['id','Minute','Team'],how='outer')
kills_structures.head(20)

merge kill and structure dataframe,we would do data cleaning latter.

monsters = monsters[ ~monsters['id'].isin(old_dragon_id)]
monsters['Type2'] = np.where( monsters['Type']=="FIRE_DRAGON", "DRAGON",
                    np.where( monsters['Type']=="EARTH_DRAGON","DRAGON",
                    np.where( monsters['Type']=="WATER_DRAGON","DRAGON",       
                    np.where( monsters['Type']=="AIR_DRAGON","DRAGON",   
                             monsters['Type']))))

monsters = monsters[ monsters['Time']>0]

monsters['Minute'] = monsters['Time'].astype(int)

monsters['Team'] = np.where( monsters['Team']=="bDragons","Blue",
                   np.where( monsters['Team']=="bHeralds","Blue",
                   np.where( monsters['Team']=="bBarons", "Blue", 
                           "Red")))



monsters.head()

kills_structures_monsters = kills_structures.merge(monsters[['id','Minute','Team','Time','Type2']], on=['id','Minute'],how='outer')
kills_structures_monsters = kills_structures_monsters.sort_values(by=['id','Minute'])
kills_structures_monsters.head(5)

merge all dataframe into one.


stackedData = killsGrouped.append(structures2)
stackedData = stackedData.append(monsters[['id','Address','Team','Minute','Time','Type2']])

stackedData['Time2'] = stackedData['Time'].fillna(stackedData['Time Avg'])

stackedData = stackedData.sort_values(by=['id','Time2'])

stackedData['EventNum'] = stackedData.groupby('id').cumcount()+1

stackedData = stackedData[['id','EventNum','Team','Minute','Time2','Count','Type','Lane','Type2']]

stackedData.columns = ['id','EventNum','Team','Minute','Time','KillCount','StructType','StructLane','Monster']

stackedData.head(5)

stackedData['Event'] = np.where(stackedData['KillCount']>0,"KILLS",None)
stackedData['Event'] = stackedData['Event'].fillna(stackedData['StructType'])
stackedData['Event'] = stackedData['Event'].fillna(stackedData['Monster'])

                        

stackedData.head(10)

By transformation We now have dataframe contains type of structure, monter and event classification on event column and count event number on Eventnum column. We now have to change them to int format for our model train by using cumcount on each group with same team and same time column which contains the string in same type of events.

stackedData.to_pickle("leagueoflegends/stacked.pkl")

stackedDatas = pd.read_pickle("leagueoflegends/stacked.pkl")

stackedDatas = stackedData.sort_values(by=['id','Minute'])

stackedDatas = stackedDatas.set_index(np.arange(len(stackedDatas.index)))

stackedDatas['Blue_Kill_count'] = stackedDatas[(stackedDatas['Event'].str.contains('KILL'))&(stackedDatas['Team'].str.contains('Blue'))].groupby(['Team','id']).cumcount()+1
stackedDatas['Blue_Object_count'] = stackedDatas[(stackedDatas['Event'].str.contains('TURRET'))&(stackedDatas['Team'].str.contains('Blue'))].groupby(['Team','id']).cumcount()+1
stackedDatas['Blue_Monster_count'] = stackedDatas[(stackedDatas['Event'].str.contains('DRAGON|HERALD|BARON'))&(stackedDatas['Team'].str.contains('Blue'))].groupby(['Team','id']).cumcount()+1
stackedDatas['Red_Kill_count'] = stackedDatas[(stackedDatas['Event'].str.contains('KILL'))&(stackedDatas['Team'].str.contains('Red'))].groupby(['Team','id']).cumcount()+1
stackedDatas['Red_Object_count'] = stackedDatas[(stackedDatas['Event'].str.contains('TURRET'))&(stackedDatas['Team'].str.contains('Red'))].groupby(['Team','id']).cumcount()+1
stackedDatas['Red_Monster_count'] = stackedDatas[(stackedDatas['Event'].str.contains('DRAGON|HERALD|BARON'))&(stackedDatas['Team'].str.contains('Red'))].groupby(['Team','id']).cumcount()+1

stackedDatas

Have so may missing value on our final dataset, we fill those data by group of same id using ffill function, finally fill those nan value with 0 because the missing value is the count which the event has not happened yet.